In [ ]:
# Surgical Video Processing Framework - Example Notebook
# This notebook demonstrates the core functionality of the framework

{
    "cells": [
        {
            "cell_type": "markdown",
            "metadata": {},
            "source": [
                "# Surgical Video Processing Framework Demo\n",
                "\n",
                "This notebook demonstrates the key features of the surgical video processing framework for phacoemulsification cataract surgery videos.\n",
                "\n",
                "## Framework Overview\n",
                "\n",
                "The framework provides:\n",
                "- **Hospital-specific processing** for Farabi and Noor hospitals\n",
                "- **Quality control** with automated assessment\n",
                "- **Privacy protection** with HIPAA-compliant de-identification\n",
                "- **Optimized compression** for surgical video characteristics",
            ],
        },
        {
            "cell_type": "code",
            "execution_count": null,
            "metadata": {},
            "outputs": [],
            "source": [
                "# Import required libraries\n",
                "import sys\n",
                "import numpy as np\n",
                "import matplotlib.pyplot as plt\n",
                "import pandas as pd\n",
                "from pathlib import Path\n",
                "\n",
                "# Add framework to path\n",
                "sys.path.append('../')\n",
                "\n",
                "# Import framework components\n",
                "from surgical_video_processing.configs import ConfigManager\n",
                "from surgical_video_processing.metadata import MetadataExtractor, MetadataManager\n",
                "from surgical_video_processing.quality_control import QualityControlPipeline\n",
                "from surgical_video_processing.pipelines import SurgicalVideoProcessor\n",
                "\n",
                'print("✅ Framework imported successfully")',
            ],
        },
        {
            "cell_type": "markdown",
            "metadata": {},
            "source": [
                "## 1. Configuration Management\n",
                "\n",
                "The framework uses YAML configurations optimized for different hospitals and use cases.",
            ],
        },
        {
            "cell_type": "code",
            "execution_count": null,
            "metadata": {},
            "outputs": [],
            "source": [
                "# Load hospital-specific configurations\n",
                'farabi_config = ConfigManager.load_config("../configs/farabi_config.yaml")\n',
                'noor_config = ConfigManager.load_config("../configs/noor_config.yaml")\n',
                "\n",
                'print("Farabi Hospital Configuration:")\n',
                "print(f\"  Target Resolution: {farabi_config['processing']['target_resolution']}\")\n",
                "print(f\"  Target FPS: {farabi_config['processing']['target_fps']}\")\n",
                "print(f\"  Quality Threshold: {farabi_config['quality_control']['min_overall_score']}\")\n",
                "\n",
                'print("\\nNoor Hospital Configuration:")\n',
                "print(f\"  Target Resolution: {noor_config['processing']['target_resolution']}\")\n",
                "print(f\"  Target FPS: {noor_config['processing']['target_fps']}\")\n",
                "print(f\"  Quality Threshold: {noor_config['quality_control']['min_overall_score']}\")",
            ],
        },
        {
            "cell_type": "markdown",
            "metadata": {},
            "source": [
                "## 2. Metadata Extraction\n",
                "\n",
                "Extract comprehensive metadata from surgical videos including automatic hospital detection.",
            ],
        },
        {
            "cell_type": "code",
            "execution_count": null,
            "metadata": {},
            "outputs": [],
            "source": [
                "# Initialize metadata extractor\n",
                "extractor = MetadataExtractor()\n",
                "\n",
                "# Example: Process sample video metadata (replace with actual video path)\n",
                'sample_video = "../test_data/sample_surgery.mp4"  # Update path as needed\n',
                "\n",
                "if Path(sample_video).exists():\n",
                "    metadata = extractor.extract_metadata(sample_video)\n",
                "    \n",
                '    print("Video Metadata:")\n',
                '    print(f"  Resolution: {metadata.width}x{metadata.height}")\n',
                '    print(f"  Frame Rate: {metadata.fps:.1f} fps")\n',
                '    print(f"  Duration: {metadata.duration_seconds:.1f} seconds")\n',
                '    print(f"  Hospital Source: {metadata.hospital_source}")\n',
                '    print(f"  Equipment: {metadata.equipment_model}")\n',
                '    print(f"  File Size: {metadata.file_size_bytes / (1024*1024):.1f} MB")\nelse:\n',
                '    print("Sample video not found. Create a test video or update the path.")\n',
                "    \n",
                "    # Create synthetic metadata for demonstration\n",
                '    print("\\nSynthetic Example - Farabi Hospital:")\n',
                '    print("  Resolution: 720x480")\n',
                '    print("  Frame Rate: 30.0 fps")\n',
                '    print("  Hospital Source: farabi")\n',
                '    print("  Equipment: Haag-Streit HS Hi-R NEO 900")\n',
                "    \n",
                '    print("\\nSynthetic Example - Noor Hospital:")\n',
                '    print("  Resolution: 1920x1080")\n',
                '    print("  Frame Rate: 60.0 fps")\n',
                '    print("  Hospital Source: noor")\n',
                '    print("  Equipment: ZEISS ARTEVO 800")',
            ],
        },
        {
            "cell_type": "markdown",
            "metadata": {},
            "source": [
                "## 3. Quality Control Analysis\n",
                "\n",
                "Demonstrate the quality control pipeline for surgical videos.",
            ],
        },
        {
            "cell_type": "code",
            "execution_count": null,
            "metadata": {},
            "outputs": [],
            "source": [
                "# Create synthetic test frames for quality analysis\n",
                'def create_test_frame(quality_type="good"):\n',
                '    """Create test frames with different quality characteristics."""\n',
                '    if quality_type == "good":\n',
                "        # Sharp, well-exposed surgical field\n",
                "        frame = np.random.randint(50, 200, (480, 720, 3), dtype=np.uint8)\n",
                "        # Add surgical instruments (high contrast)\n",
                "        frame[200:280, 300:400] = [255, 255, 255]  # Bright instrument\n",
                "        frame[250:270, 320:380] = [0, 0, 0]        # Shadow for contrast\n",
                '    elif quality_type == "blurry":\n',
                "        # Blurry frame with low frequency content\n",
                "        frame = np.random.randint(80, 120, (480, 720, 3), dtype=np.uint8)\n",
                "        # Apply blur to simulate poor focus\n",
                "        import cv2\n",
                "        frame = cv2.GaussianBlur(frame, (15, 15), 5)\n",
                '    elif quality_type == "glare":\n',
                "        # Frame with excessive glare\n",
                "        frame = np.random.randint(50, 100, (480, 720, 3), dtype=np.uint8)\n",
                "        frame[100:200, 200:400] = [255, 255, 255]  # Large glare region\n",
                "    else:\n",
                "        frame = np.random.randint(0, 255, (480, 720, 3), dtype=np.uint8)\n",
                "    \n",
                "    return frame\n",
                "\n",
                "# Create test frames\n",
                'good_frame = create_test_frame("good")\n',
                'blurry_frame = create_test_frame("blurry")\n',
                'glare_frame = create_test_frame("glare")\n',
                "\n",
                "# Initialize quality control pipeline\n",
                'quality_config = ConfigManager.load_config("../configs/default.yaml")\n',
                "quality_pipeline = QualityControlPipeline(quality_config)\n",
                "\n",
                "# Analyze frame quality\n",
                'print("Quality Analysis Results:")\n',
                "\n",
                "# Focus analysis\n",
                "good_focus = quality_pipeline.focus_checker.analyze_focus(good_frame)\n",
                "blurry_focus = quality_pipeline.focus_checker.analyze_focus(blurry_frame)\n",
                'print(f"\\nFocus Scores:")\n',
                'print(f"  Good frame: {good_focus:.2f}")\n',
                'print(f"  Blurry frame: {blurry_focus:.2f}")\n',
                "\n",
                "# Glare detection\n",
                "normal_glare = quality_pipeline.glare_detector.detect_glare(good_frame)\n",
                "high_glare = quality_pipeline.glare_detector.detect_glare(glare_frame)\n",
                'print(f"\\nGlare Detection:")\n',
                'print(f"  Normal frame: {normal_glare:.1f}% glare")\n',
                'print(f"  Glare frame: {high_glare:.1f}% glare")',
            ],
        },
        {
            "cell_type": "markdown",
            "metadata": {},
            "source": [
                "## 4. Visualization\n",
                "\n",
                "Visualize quality analysis results and frame comparisons.",
            ],
        },
        {
            "cell_type": "code",
            "execution_count": null,
            "metadata": {},
            "outputs": [],
            "source": [
                "# Visualize test frames and quality metrics\n",
                "fig, axes = plt.subplots(2, 3, figsize=(15, 10))\n",
                "\n",
                "# Display frames\n",
                "axes[0,0].imshow(good_frame)\n",
                "axes[0,0].set_title('Good Quality Frame')\n",
                "axes[0,0].axis('off')\n",
                "\n",
                "axes[0,1].imshow(blurry_frame)\n",
                "axes[0,1].set_title('Blurry Frame')\n",
                "axes[0,1].axis('off')\n",
                "\n",
                "axes[0,2].imshow(glare_frame)\n",
                "axes[0,2].set_title('Frame with Glare')\n",
                "axes[0,2].axis('off')\n",
                "\n",
                "# Quality metrics comparison\n",
                "quality_metrics = {\n",
                "    'Frame Type': ['Good', 'Blurry', 'Glare'],\n",
                "    'Focus Score': [good_focus, blurry_focus, quality_pipeline.focus_checker.analyze_focus(glare_frame)],\n",
                "    'Glare %': [normal_glare, quality_pipeline.glare_detector.detect_glare(blurry_frame), high_glare]\n",
                "}\n",
                "\n",
                "# Bar charts for metrics\n",
                "x_pos = np.arange(len(quality_metrics['Frame Type']))\n",
                "\n",
                "axes[1,0].bar(x_pos, quality_metrics['Focus Score'], color=['green', 'orange', 'red'])\n",
                "axes[1,0].set_title('Focus Score Comparison')\n",
                "axes[1,0].set_xticks(x_pos)\n",
                "axes[1,0].set_xticklabels(quality_metrics['Frame Type'])\n",
                "axes[1,0].set_ylabel('Focus Score')\n",
                "\n",
                "axes[1,1].bar(x_pos, quality_metrics['Glare %'], color=['green', 'orange', 'red'])\n",
                "axes[1,1].set_title('Glare Percentage Comparison')\n",
                "axes[1,1].set_xticks(x_pos)\n",
                "axes[1,1].set_xticklabels(quality_metrics['Frame Type'])\n",
                "axes[1,1].set_ylabel('Glare %')\n",
                "\n",
                "# Quality threshold visualization\n",
                "threshold_focus = quality_config['quality_control']['min_focus_score']\n",
                "threshold_glare = quality_config['quality_control']['max_glare_percentage']\n",
                "\n",
                "axes[1,0].axhline(y=threshold_focus, color='red', linestyle='--', label=f'Threshold: {threshold_focus}')\n",
                "axes[1,1].axhline(y=threshold_glare, color='red', linestyle='--', label=f'Threshold: {threshold_glare}')\n",
                "\n",
                "axes[1,0].legend()\n",
                "axes[1,1].legend()\n",
                "\n",
                "# Overall quality assessment\n",
                "passed_quality = []\n",
                "for i, frame_type in enumerate(quality_metrics['Frame Type']):\n",
                "    focus_pass = quality_metrics['Focus Score'][i] >= threshold_focus\n",
                "    glare_pass = quality_metrics['Glare %'][i] <= threshold_glare\n",
                "    overall_pass = focus_pass and glare_pass\n",
                "    passed_quality.append('PASS' if overall_pass else 'FAIL')\n",
                "\n",
                "colors = ['green' if p == 'PASS' else 'red' for p in passed_quality]\n",
                "axes[1,2].bar(x_pos, [1]*len(x_pos), color=colors)\n",
                "axes[1,2].set_title('Overall Quality Assessment')\n",
                "axes[1,2].set_xticks(x_pos)\n",
                "axes[1,2].set_xticklabels([f'{t}\\n{p}' for t, p in zip(quality_metrics['Frame Type'], passed_quality)])\n",
                "axes[1,2].set_ylabel('Quality Status')\n",
                "axes[1,2].set_ylim(0, 1.2)\n",
                "\n",
                "plt.tight_layout()\n",
                "plt.show()\n",
                "\n",
                'print("\\nQuality Assessment Summary:")\n',
                "for i, frame_type in enumerate(quality_metrics['Frame Type']):\n",
                "    print(f\"  {frame_type}: {passed_quality[i]} (Focus: {quality_metrics['Focus Score'][i]:.2f}, Glare: {quality_metrics['Glare %'][i]:.1f}%)\")",
            ],
        },
        {
            "cell_type": "markdown",
            "metadata": {},
            "source": [
                "## 5. Hospital-Specific Processing\n",
                "\n",
                "Demonstrate different processing approaches for each hospital.",
            ],
        },
        {
            "cell_type": "code",
            "execution_count": null,
            "metadata": {},
            "outputs": [],
            "source": [
                "# Compare hospital-specific configurations\n",
                "hospitals = {\n",
                "    'Farabi': {\n",
                "        'config': farabi_config,\n",
                "        'equipment': 'Haag-Streit HS Hi-R NEO 900',\n",
                "        'resolution': '720×480',\n",
                "        'fps': '30'\n",
                "    },\n",
                "    'Noor': {\n",
                "        'config': noor_config,\n",
                "        'equipment': 'ZEISS ARTEVO 800',\n",
                "        'resolution': '1920×1080', \n",
                "        'fps': '60'\n",
                "    }\n",
                "}\n",
                "\n",
                'print("Hospital-Specific Processing Configurations:")\n',
                'print("=" * 60)\n',
                "\n",
                "for hospital, info in hospitals.items():\n",
                "    config = info['config']\n",
                '    print(f"\\n{hospital} Hospital:")\n',
                "    print(f\"  Equipment: {info['equipment']}\")\n",
                "    print(f\"  Native Resolution: {info['resolution']}\")\n",
                "    print(f\"  Native FPS: {info['fps']}\")\n",
                "    print(f\"  Quality Threshold: {config['quality_control']['min_overall_score']}\")\n",
                "    print(f\"  Compression CRF: {config['compression']['crf_value']}\")\n",
                "    print(f\"  Noise Reduction: {config['preprocessing']['noise_reduction_strength']}\")\n",
                "    print(f\"  GPU Acceleration: {config['environment']['gpu_acceleration']}\")\n",
                "\n",
                "# Visualization of configuration differences\n",
                "fig, axes = plt.subplots(1, 3, figsize=(15, 5))\n",
                "\n",
                "# Quality thresholds\n",
                "quality_thresholds = [hospitals['Farabi']['config']['quality_control']['min_overall_score'],\n",
                "                     hospitals['Noor']['config']['quality_control']['min_overall_score']]\n",
                "axes[0].bar(['Farabi', 'Noor'], quality_thresholds, color=['orange', 'blue'])\n",
                "axes[0].set_title('Quality Thresholds')\n",
                "axes[0].set_ylabel('Minimum Score')\n",
                "\n",
                "# Compression settings\n",
                "crf_values = [hospitals['Farabi']['config']['compression']['crf_value'],\n",
                "              hospitals['Noor']['config']['compression']['crf_value']]\n",
                "axes[1].bar(['Farabi', 'Noor'], crf_values, color=['orange', 'blue'])\n",
                "axes[1].set_title('Compression CRF Values')\n",
                "axes[1].set_ylabel('CRF Value (lower = higher quality)')\n",
                "\n",
                "# Processing approach\n",
                "noise_reduction = [hospitals['Farabi']['config']['preprocessing']['noise_reduction_strength'],\n",
                "                   hospitals['Noor']['config']['preprocessing']['noise_reduction_strength']]\n",
                "axes[2].bar(['Farabi', 'Noor'], noise_reduction, color=['orange', 'blue'])\n",
                "axes[2].set_title('Noise Reduction Strength')\n",
                "axes[2].set_ylabel('Strength Level')\n",
                "\n",
                "plt.tight_layout()\n",
                "plt.show()",
            ],
        },
        {
            "cell_type": "markdown",
            "metadata": {},
            "source": [
                "## 6. Framework Usage Examples\n",
                "\n",
                "Practical examples of using the framework for different scenarios.",
            ],
        },
        {
            "cell_type": "code",
            "execution_count": null,
            "metadata": {},
            "outputs": [],
            "source": [
                "# Example 1: Single video processing\n",
                'print("Example 1: Single Video Processing")\n',
                'print("-" * 40)\n',
                'print("# Process a single video from Farabi Hospital")\n',
                'print("python main.py --hospital farabi --input surgery_001.mp4 --output ./processed")\n',
                "print()\n",
                "\n",
                "# Example 2: Batch processing\n",
                'print("Example 2: Batch Processing")\n',
                'print("-" * 40)\n',
                'print("# Process all videos in a directory")\n',
                'print("python main.py --hospital noor --input ./noor_videos/ --output ./processed_noor/")\n',
                "print()\n",
                "\n",
                "# Example 3: Quick processing for large datasets\n",
                'print("Example 3: Quick Processing Mode")\n',
                'print("-" * 40)\n',
                'print("# Fast processing for large datasets")\n',
                'print("python main.py --quick --input ./large_dataset/ --output ./quick_processed/")\n',
                "print()\n",
                "\n",
                "# Example 4: High quality for research\n",
                'print("Example 4: High Quality Research Mode")\n',
                'print("-" * 40)\n',
                'print("# Maximum quality for research applications")\n',
                'print("python main.py --high-quality --input ./research_videos/ --output ./research_processed/")\n',
                "print()\n",
                "\n",
                "# Example 5: Custom configuration\n",
                'print("Example 5: Custom Configuration")\n',
                'print("-" * 40)\n',
                'print("# Use custom configuration file")\n',
                'print("python main.py --config my_custom_config.yaml --input ./videos/ --output ./processed/")\n',
                "print()\n",
                "\n",
                "# Example 6: Python API usage\n",
                'print("Example 6: Python API Usage")\n',
                'print("-" * 40)\n',
                "code_example = '''\n",
                "from surgical_video_processing.configs import ConfigManager\n",
                "from surgical_video_processing.pipelines import SurgicalVideoProcessor\n",
                "\n",
                "# Load configuration\n",
                'config = ConfigManager.load_config("farabi_config.yaml")\n',
                "\n",
                "# Initialize processor\n",
                "processor = SurgicalVideoProcessor(config)\n",
                "\n",
                "# Process video\n",
                "result = processor.process_video(\n",
                '    input_path="surgery_video.mp4",\n',
                '    output_dir="./processed"\n',
                ")\n",
                "\n",
                "if result.success:\n",
                '    print(f"Processed: {result.output_path}")\n',
                '    print(f"Quality: {result.quality_score:.1f}")\n',
                '    print(f"Time: {result.processing_time:.1f}s")\n',
                "'''\n",
                "print(code_example)",
            ],
        },
        {
            "cell_type": "markdown",
            "metadata": {},
            "source": [
                "## 7. Performance and Statistics\n",
                "\n",
                "Analyze framework performance and processing statistics.",
            ],
        },
        {
            "cell_type": "code",
            "execution_count": null,
            "metadata": {},
            "outputs": [],
            "source": [
                "# Simulate processing statistics for demonstration\n",
                "import random\n",
                "random.seed(42)\n",
                "\n",
                "# Generate synthetic processing data\n",
                "def generate_processing_stats(hospital, num_videos=100):\n",
                "    stats = []\n",
                "    \n",
                "    for i in range(num_videos):\n",
                "        if hospital == 'farabi':\n",
                "            # Farabi characteristics (720p, 30fps)\n",
                "            duration = random.uniform(8, 25)  # 8-25 minute surgeries\n",
                "            quality_score = random.uniform(60, 90)  # Lower quality source\n",
                "            processing_time = duration * random.uniform(0.3, 0.8)  # Processing speed\n",
                "            file_size_mb = duration * random.uniform(15, 25)  # File size\n",
                "        else:\n",
                "            # Noor characteristics (1080p, 60fps)\n",
                "            duration = random.uniform(10, 30)  # 10-30 minute surgeries\n",
                "            quality_score = random.uniform(75, 95)  # Higher quality source\n",
                "            processing_time = duration * random.uniform(0.5, 1.2)  # Processing speed\n",
                "            file_size_mb = duration * random.uniform(45, 75)  # Larger file size\n",
                "        \n",
                "        stats.append({\n",
                "            'hospital': hospital,\n",
                "            'video_id': f\"{hospital}_{i+1:03d}\",\n",
                "            'duration_min': duration,\n",
                "            'quality_score': quality_score,\n",
                "            'processing_time_min': processing_time,\n",
                "            'file_size_mb': file_size_mb,\n",
                "            'success': random.random() > 0.05  # 95% success rate\n",
                "        })\n",
                "    \n",
                "    return stats\n",
                "\n",
                "# Generate data for both hospitals\n",
                "farabi_stats = generate_processing_stats('farabi', 150)\n",
                "noor_stats = generate_processing_stats('noor', 150)\n",
                "all_stats = farabi_stats + noor_stats\n",
                "\n",
                "# Convert to DataFrame for analysis\n",
                "df = pd.DataFrame(all_stats)\n",
                "\n",
                "# Display summary statistics\n",
                'print("Processing Statistics Summary:")\n',
                'print("=" * 50)\n',
                "summary = df.groupby('hospital').agg({\n",
                "    'duration_min': ['mean', 'std'],\n",
                "    'quality_score': ['mean', 'std'],\n",
                "    'processing_time_min': ['mean', 'std'],\n",
                "    'file_size_mb': ['mean', 'std'],\n",
                "    'success': 'mean'\n",
                "}).round(2)\n",
                "\n",
                "print(summary)\n",
                "\n",
                "# Visualize statistics\n",
                "fig, axes = plt.subplots(2, 3, figsize=(18, 12))\n",
                "\n",
                "# Duration distribution\n",
                "df[df['hospital'] == 'farabi']['duration_min'].hist(alpha=0.7, label='Farabi', bins=20, ax=axes[0,0])\n",
                "df[df['hospital'] == 'noor']['duration_min'].hist(alpha=0.7, label='Noor', bins=20, ax=axes[0,0])\n",
                "axes[0,0].set_title('Surgery Duration Distribution')\n",
                "axes[0,0].set_xlabel('Duration (minutes)')\n",
                "axes[0,0].legend()\n",
                "\n",
                "# Quality score distribution\n",
                "df[df['hospital'] == 'farabi']['quality_score'].hist(alpha=0.7, label='Farabi', bins=20, ax=axes[0,1])\n",
                "df[df['hospital'] == 'noor']['quality_score'].hist(alpha=0.7, label='Noor', bins=20, ax=axes[0,1])\n",
                "axes[0,1].set_title('Quality Score Distribution')\n",
                "axes[0,1].set_xlabel('Quality Score')\n",
                "axes[0,1].legend()\n",
                "\n",
                "# Processing time vs duration\n",
                "farabi_data = df[df['hospital'] == 'farabi']\n",
                "noor_data = df[df['hospital'] == 'noor']\n",
                "axes[0,2].scatter(farabi_data['duration_min'], farabi_data['processing_time_min'], \n",
                "                 alpha=0.6, label='Farabi', s=30)\n",
                "axes[0,2].scatter(noor_data['duration_min'], noor_data['processing_time_min'], \n",
                "                 alpha=0.6, label='Noor', s=30)\n",
                "axes[0,2].set_title('Processing Time vs Duration')\n",
                "axes[0,2].set_xlabel('Video Duration (min)')\n",
                "axes[0,2].set_ylabel('Processing Time (min)')\n",
                "axes[0,2].legend()\n",
                "\n",
                "# File size distribution\n",
                "df[df['hospital'] == 'farabi']['file_size_mb'].hist(alpha=0.7, label='Farabi', bins=20, ax=axes[1,0])\n",
                "df[df['hospital'] == 'noor']['file_size_mb'].hist(alpha=0.7, label='Noor', bins=20, ax=axes[1,0])\n",
                "axes[1,0].set_title('File Size Distribution')\n",
                "axes[1,0].set_xlabel('File Size (MB)')\n",
                "axes[1,0].legend()\n",
                "\n",
                "# Success rate comparison\n",
                "success_rates = df.groupby('hospital')['success'].mean()\n",
                "axes[1,1].bar(success_rates.index, success_rates.values * 100, color=['orange', 'blue'])\n",
                "axes[1,1].set_title('Processing Success Rate')\n",
                "axes[1,1].set_ylabel('Success Rate (%)')\n",
                "axes[1,1].set_ylim(90, 100)\n",
                "\n",
                "# Processing efficiency (duration/processing_time)\n",
                "df['efficiency'] = df['duration_min'] / df['processing_time_min']\n",
                "efficiency_by_hospital = df.groupby('hospital')['efficiency'].mean()\n",
                "axes[1,2].bar(efficiency_by_hospital.index, efficiency_by_hospital.values, color=['orange', 'blue'])\n",
                "axes[1,2].set_title('Processing Efficiency')\n",
                "axes[1,2].set_ylabel('Efficiency Ratio (higher = faster)')\n",
                "\n",
                "plt.tight_layout()\n",
                "plt.show()\n",
                "\n",
                "# Print key insights\n",
                'print("\\nKey Insights:")\n',
                'print(f"• Total videos processed: {len(df)}")\n',
                "print(f\"• Overall success rate: {df['success'].mean()*100:.1f}%\")\n",
                "print(f\"• Average quality score: {df['quality_score'].mean():.1f}\")\n",
                "print(f\"• Farabi avg processing efficiency: {efficiency_by_hospital['farabi']:.2f}x\")\n",
                "print(f\"• Noor avg processing efficiency: {efficiency_by_hospital['noor']:.2f}x\")",
            ],
        },
        {
            "cell_type": "markdown",
            "metadata": {},
            "source": [
                "## 8. Next Steps and Resources\n",
                "\n",
                "This notebook has demonstrated the core functionality of the surgical video processing framework.",
            ],
        },
        {
            "cell_type": "code",
            "execution_count": null,
            "metadata": {},
            "outputs": [],
            "source": [
                'print("🎯 Next Steps:")\n',
                'print("1. Test with your actual surgical videos")\n',
                'print("2. Customize configurations for your specific needs")\n',
                'print("3. Set up batch processing workflows")\n',
                'print("4. Integrate quality control into your pipeline")\n',
                'print("5. Implement automated de-identification procedures")\n',
                "print()\n",
                'print("📚 Additional Resources:")\n',
                'print("• Framework Documentation: README.md")\n',
                'print("• Configuration Guide: configs/README.md")\n',
                'print("• Testing Suite: tests/README.md")\n',
                'print("• Utility Scripts: scripts/README.md")\n',
                'print("• Metadata Management: metadata/README.md")\n',
                "print()\n",
                'print("🔧 Command Line Examples:")\n',
                'print("# Setup system")\n',
                'print("python scripts/utilities.py setup")\n',
                "print()\n",
                'print("# Process Farabi hospital videos")\n',
                'print("python main.py --hospital farabi --input ./videos --output ./processed")\n',
                "print()\n",
                'print("# Run system tests")\n',
                'print("python -m surgical_video_processing.tests")\n',
                "print()\n",
                'print("# Analyze dataset")\n',
                'print("python scripts/utilities.py analyze-dataset --input ./videos")\n',
                "print()\n",
                'print("✅ Framework is ready for production use!")',
            ],
        },
    ],
    "metadata": {
        "kernelspec": {
            "display_name": "Python 3",
            "language": "python",
            "name": "python3",
        },
        "language_info": {
            "codemirror_mode": {"name": "ipython", "version": 3},
            "file_extension": ".py",
            "mimetype": "text/x-python",
            "name": "python",
            "nbconvert_exporter": "python",
            "pygments_lexer": "ipython3",
            "version": "3.8.10",
        },
    },
    "nbformat": 4,
    "nbformat_minor": 4,
}